In [4]:
import requests
import pandas as pd
import folium
from geopy.distance import geodesic
from autobahn.autobahn import get_warnings

url_bab = "https://verkehr.autobahn.de/o/autobahn"

warnings_a7 = get_warnings('A7')

# warnings_df

In [13]:
osrm_url = "http://router.project-osrm.org/route/v1/driving/"
start = "9.446996,54.793743"  # Flensburg
end = "9.479746,51.312711"  # Kassel
route_url = f"{osrm_url}{start};{end}?overview=full&geometries=geojson"
route_response = requests.get(route_url)
route_data = route_response.json()
route_coords = route_data['routes'][0]['geometry']['coordinates']
route_df = pd.DataFrame(route_coords, columns=['long', 'lat'])
# route_df

### OSM Data

In [23]:
print(f"distance: {route_data['routes'][0]['distance'] / 1000:.2f} km")
print(f"duration: {route_data['routes'][0]['duration'] / 3600:.2f} h")

distance: 469.81 km
duration: 4.92 h


In [3]:
class Car:
    def __init__(self, start_location):
        self.location = start_location
        self.route = []
        self.distance_traveled = 0.0

    def drive_to(self, destination):
        self.route.append(destination)
        self.distance_traveled += geodesic(self.location, destination).kilometers
        self.location = destination

car = Car(start_location=(54.793743, 9.446996))  # Flensburg
for coord in route_coords:
    car.drive_to((coord[1], coord[0]))

car.distance_traveled

In [4]:
m = folium.Map(location=[route_df['lat'].mean(), route_df['long'].mean()], zoom_start=6)
folium.PolyLine(route_coords, color='blue', weight=2.5, opacity=1).add_to(m)
m